In [20]:
import os
import tensorflow as tf
from tensorflow.keras import regularizers, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [6]:
# Load Data
data = pd.read_csv("../Training_Data/balanced_soldier_data.csv")

In [8]:
data.head()

,Temperature,Moisture,Water_Content,SpO2,Fatigue,Drowsiness,Stress,Heart_Rate,Respiration_Rate,Systolic_BP,Diastolic_BP,Efficiency
0,35.383229,65.701296,74.896995,96.144655,0.208223,0.078256,0.167958,66.431073,15.907304,119,78,0.726903
1,36.028752,69.182477,87.289935,99.285356,0.186550,0.138735,0.008335,82.207135,17.644151,113,76,0.775944
2,37.004390,38.142130,51.690862,88.401253,0.795877,0.860479,0.821897,115.268466,25.442539,137,88,0.216759
3,35.926560,56.353209,86.145719,97.281401,0.074097,0.278761,0.125591,86.724807,15.603473,117,70,0.719477
4,36.231738,46.817432,48.037744,89.566148,0.779676,0.624617,0.829127,118.251208,23.152386,135,85,0.247567


In [9]:
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature       5100 non-null   float64
 1   Moisture          5100 non-null   float64
 2   Water_Content     5100 non-null   float64
 3   SpO2              5100 non-null   float64
 4   Fatigue           5100 non-null   float64
 5   Drowsiness        5100 non-null   float64
 6   Stress            5100 non-null   float64
 7   Heart_Rate        5100 non-null   float64
 8   Respiration_Rate  5100 non-null   float64
 9   Systolic_BP       5100 non-null   int64  
 10  Diastolic_BP      5100 non-null   int64  
 11  Efficiency        5100 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 478.3 KB
None
       Temperature     Moisture  Water_Content         SpO2      Fatigue  \
count  5100.000000  5100.000000    5100.000000  5100.000000  5100.000000   
mean     36.824245    49.

In [11]:
X = data.drop(columns=["Efficiency"])
y = data["Efficiency"]

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [15]:
# %% Build Model
def build_model(input_shape):
    model = Sequential(
        [
            layers.Input(shape=(input_shape,)),
            layers.Dense(
                128, activation="relu", kernel_regularizer=regularizers.l2(0.01)
            ),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(
                256, activation="relu", kernel_regularizer=regularizers.l2(0.01)
            ),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(128, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(1, activation="linear"),
        ]
    )
    return model

In [16]:
model = build_model(X_train.shape[1])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,361 (310.00 KB)

 Trainable params: 78,593 (307.00 KB)

 Non-trainable params: 768 (3.00 KB)

In [18]:
# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mae"])

In [19]:
# %% Train Model (Uncomment this to train)
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True
        )
    ],
)

Epoch 1/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.5665 - mae: 0.2600 - val_loss: 0.6941 - val_mae: 0.1925
Epoch 2/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5506 - mae: 0.0978 - val_loss: 0.3367 - val_mae: 0.1727
Epoch 3/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2560 - mae: 0.0747 - val_loss: 0.1882 - val_mae: 0.1807
Epoch 4/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1218 - mae: 0.0712 - val_loss: 0.0928 - val_mae: 0.1319
Epoch 5/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0626 - mae: 0.0642 - val_loss: 0.1082 - val_mae: 0.2517
Epoch 6/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0384 - mae: 0.0628 - val_loss: 0.0441 - val_mae: 0.1181
Epoch 7/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0258 - mae: 0.0570 - val_loss: 0.0720 - val_mae: 0.2006
Epoch 8/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0217 - mae: 0.0580 - val_loss: 0.0451 - val_mae: 0.1467
Epoch 9/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

In [21]:
Trained_Model_Path = "../../backend/Trained_Models"

In [22]:
model.save(os.path.join(Trained_Model_Path,"Efficiency_Model.keras"))

In [24]:
import joblib

# Save the fitted scaler
joblib.dump(scaler, os.path.join(Trained_Model_Path,"Efficiency_Scaler.pkl"))
print("Scaler saved successfully.")

Scaler saved successfully.
